In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
import cv2
from collections import OrderedDict
import torch
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
from matplotlib import pyplot as plt
from anomalib.pre_processing.pre_process import get_transforms
import torchvision.models.detection as detection
from anomalib.data import InferenceDataset
from torch.utils.data import DataLoader
from anomalib.pre_processing import PreProcessor

from anomalib.models.rbad.region_extractor import RegionExtractor as RegionExtractor2
from anomalib.models.rbad.region import RegionExtractor as RegionExtractor1

filename = "150.tif"
image = cv2.imread(filename)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
# Transformations.
transforms = get_transforms(config=A.Compose([A.Normalize(mean=0.0, std=1.0), ToTensorV2()]))
pre_process = PreProcessor(config=transforms)

# Get the data via dataloader
dataset = InferenceDataset(path=filename, pre_process=pre_process)
dataloader = DataLoader(dataset)
i, data = next(enumerate(dataloader))

# Create the region extractor.
stage="rpn"
use_original = True
region_extractor1 = RegionExtractor1(stage=stage, use_original=use_original).eval().cuda()
region_extractor2 = RegionExtractor2(stage=stage, use_original=use_original).eval().cuda()

# Forward-Pass the input
out1 = region_extractor1([image])
out2 = region_extractor2(data["image"].cuda())

In [18]:
out1

[array([[ 77.919365 ,  96.48182  ,  89.83704  , 128.9149   ],
        [115.46756  ,  82.48316  , 126.88628  , 109.9148   ],
        [203.38243  ,  48.828087 , 211.18283  ,  67.71943  ],
        [166.53041  ,  37.683304 , 173.75429  ,  56.55531  ],
        [135.94873  ,  42.22176  , 143.47353  ,  62.55404  ],
        [128.62746  ,  84.20132  , 139.80513  , 107.595146 ],
        [ 91.97258  ,  60.271667 , 101.65103  ,  81.26704  ],
        [155.34999  ,  58.688763 , 163.23871  ,  81.62608  ],
        [144.49686  ,  58.143917 , 153.59395  ,  80.6864   ],
        [  1.6176828, 133.48872  ,  12.090294 , 158.       ],
        [116.0117   ,  86.902084 , 122.413864 ,  95.00669  ],
        [129.58817  ,  97.38628  , 135.42203  , 112.01805  ],
        [129.29918  ,  96.54453  , 136.47055  , 112.37445  ],
        [203.97433  ,  56.81103  , 209.38437  ,  67.685005 ],
        [130.07632  ,  88.14842  , 135.91035  ,  94.70936  ],
        [136.83748  ,  44.015156 , 142.04861  ,  51.518982 ],
        

In [17]:
out2[0].shape

torch.Size([24, 4])

In [ ]:
torch.allclose(out1[0], out2[0], rtol=1e-3)

In [ ]:
cv2_images = [image]
input_list = []
for cv2_image in cv2_images:
    cv2_image = cv2_image.astype(np.float32)
    cv2_image /= 255.0
    cv2_image = torch.from_numpy(cv2_image).contiguous()
    cv2_image = cv2_image.permute(2, 0, 1)
    input_list.append(cv2_image.cuda())

original_image_sizes = [img.shape[-2:] for img in input_list]
images, targets = region_extractor1.transform(input_list)
new_image_sizes = images.image_sizes

features = region_extractor1.backbone(images.tensors)
if isinstance(features, torch.Tensor):
    features = OrderedDict([(0, features)])

proposals, proposal_losses = region_extractor1.rpn(images, features, targets)    

In [ ]:
original_image_sizes

In [ ]:
input_tensor = data["image"].cuda()
images = [image for image in input_tensor]

original_images_sizes2 = [image.shape[-2:] for image in images]
original_images_sizes2